In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

22/07/22 01:01:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/22 01:01:14 WARN MetricsSystem: Using default name SparkStatusTracker for source because neither spark.metrics.namespace nor spark.app.id is set.


In [2]:
from IPython.core.magic import line_magic, line_cell_magic, Magics, magics_class

@magics_class
class DatabricksConnectMagics(Magics):

   @line_cell_magic
   def sql(self, line, cell=None):
       if cell and line:
           raise ValueError("Line must be empty for cell magic", line)
       try:
           from autovizwidget.widget.utils import display_dataframe
       except ImportError:
           print("Please run `pip install autovizwidget` to enable the visualization widget.")
           display_dataframe = lambda x: x
       return display_dataframe(self.get_spark().sql(cell or line).toPandas())

   def get_spark(self):
       user_ns = get_ipython().user_ns
       if "spark" in user_ns:
           return user_ns["spark"]
       else:
           from pyspark.sql import SparkSession
           user_ns["spark"] = SparkSession.builder.getOrCreate()
           return user_ns["spark"]

ip = get_ipython()
ip.register_magics(DatabricksConnectMagics)

In [3]:
from pyspark.sql.types import *
from datetime import date

In [4]:
# Create a Spark DataFrame consisting of high and low temperatures
# by airport code and date.
schema = StructType([
    StructField('AirportCode', StringType(), False),
    StructField('Date', DateType(), False),
    StructField('TempHighF', IntegerType(), False),
    StructField('TempLowF', IntegerType(), False)
])

data = [
    [ 'BLI', date(2021, 4, 3), 52, 43],
    [ 'BLI', date(2021, 4, 2), 50, 38],
    [ 'BLI', date(2021, 4, 1), 52, 41],
    [ 'PDX', date(2021, 4, 3), 64, 45],
    [ 'PDX', date(2021, 4, 2), 61, 41],
    [ 'PDX', date(2021, 4, 1), 66, 39],
    [ 'SEA', date(2021, 4, 3), 57, 43],
    [ 'SEA', date(2021, 4, 2), 54, 39],
    [ 'SEA', date(2021, 4, 1), 56, 41]
]

temps = spark.createDataFrame(data, schema)

In [5]:
# Create a table on the Databricks cluster and then fill
# the table with the DataFrame's contents.
# If the table already exists from a previous run,
# delete it first.
spark.sql('USE christopherchalcraft_scratch')
spark.sql('DROP TABLE IF EXISTS demo_temps_table')
temps.write.saveAsTable('demo_temps_table')

View job details at https://e2-demo-field-eng.cloud.databricks.com/?o=547448167446934#/setting/clusters/0722-070458-t24q3fgi/sparkUi
View job details at https://e2-demo-field-eng.cloud.databricks.com/?o=547448167446934#/setting/clusters/0722-070458-t24q3fgi/sparkUi


In [6]:
# Query the table on the Databricks cluster, returning rows
# where the airport code is not BLI and the date is later
# than 2021-04-01. Group the results and order by high
# temperature in descending order.
df_temps = spark.sql("SELECT * FROM demo_temps_table " \
    "WHERE AirportCode != 'BLI' AND Date > '2021-04-01' " \
    "GROUP BY AirportCode, Date, TempHighF, TempLowF " \
    "ORDER BY TempHighF DESC")
df_temps.show()

View job details at https://e2-demo-field-eng.cloud.databricks.com/?o=547448167446934#/setting/clusters/0722-070458-t24q3fgi/sparkUi


+-----------+----------+---------+--------+
|AirportCode|      Date|TempHighF|TempLowF|
+-----------+----------+---------+--------+
|        PDX|2021-04-03|       64|      45|
|        PDX|2021-04-02|       61|      41|
|        SEA|2021-04-03|       57|      43|
|        SEA|2021-04-02|       54|      39|
+-----------+----------+---------+--------+



In [9]:
df_temps.write.format("delta").mode("overwrite").save("/tmp/christopher.chalcraft/temp_delta")

View job details at https://e2-demo-field-eng.cloud.databricks.com/?o=547448167446934#/setting/clusters/0722-070458-t24q3fgi/sparkUi


In [11]:
spark.sql("""
CREATE TABLE temp_delta
USING DELTA
LOCATION '/tmp/christopher.chalcraft/temp_delta'
  """
         )

View job details at https://e2-demo-field-eng.cloud.databricks.com/?o=547448167446934#/setting/clusters/0722-070458-t24q3fgi/sparkUi


DataFrame[]

In [18]:
spark.sql("show tblproperties temp_delta")

DataFrame[key: string, value: string]

In [17]:
%sql show tblproperties temp_delta

View job details at https://e2-demo-field-eng.cloud.databricks.com/?o=547448167446934#/setting/clusters/0722-070458-t24q3fgi/sparkUi
View job details at https://e2-demo-field-eng.cloud.databricks.com/?o=547448167446934#/setting/clusters/0722-070458-t24q3fgi/sparkUi


Output()

AutoVizWidget()